In [ ]:
#安裝套件
!pip install pyserini

In [ ]:
import json
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import pandas as pd
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IR/wiki_article_list_2023_tra.json', sep="\n#", header=None)
data


<ipython-input-30-c0dfaad3dd5e>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IR/wiki_article_list_2023_tra.json', sep="\n#", header=None)


,0
0,[
1,"""引渡是一個國際執法的概念，指一個司法管轄區將在另一個司法管轄區被指控或被定罪的人，移交給當..."
2,"""波爾·安德森PoulWilliamAnderson，1926年11月25日-2001年7月..."
3,"""彭國，又稱大彭國大彭氏大彭氏國，是中國上古時的一個方國，位於今江蘇徐州附近。傳說由彭氏部落..."
4,"""亞伯拉罕，意為“多國之父”，原名作亞伯蘭希伯來語：或亞巴郎Abram，意為“崇高之父”，是..."
...,...
1323052,"""石板屋是臺灣原住民魯凱族，排灣族及少數布農族和泰雅族用石板建造的傳統式住屋。相傳以前下雨時..."
1323053,"""農業部是一個主管農業的部或政府機構，其首長通常稱為農業部部長。農業部的職責通常是制定行政法..."
1323054,"""阿馬，，英國北愛爾蘭阿馬郡阿馬區的城市，北愛爾蘭五座城市之一。羅馬天主教和聖公會的全愛爾蘭..."
1323055,"""夜視儀，簡稱NVD，也稱夜視鏡，是透過光學科技強化黑暗中光源的科技裝置，通常用於軍事用途。..."


In [ ]:
import json
import os.path
from tqdm import tqdm  # 導入tqdm

# 逐行處理並保存為JSON檔案，跳過第一行和最後一行
# 使用tqdm包裹data.iterrows()來顯示進度條
for index, row in tqdm(data.iterrows(), total=data.shape[0]):
    if index == 0 or index == len(data) - 1:
        continue  # 跳過第一行和最後一行
    # 創建一個字典來存儲id和contents
    json_obj = {
        "id": index,
        "contents": row[0]
    }
    # 指定要寫入的文件路徑
    file_path = 'tests/resources/'+ str(index) +'.json'
    # 確保資料夾存在，如果不存在則創建
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    # 將字典寫入到JSON文件
    with open(file_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_obj, json_file, ensure_ascii=False)


100%|██████████| 1323057/1323057 [09:04<00:00, 2430.62it/s]


1. 使用 Python 命令執行 Pyserini 中的 index.lucene 模組
1. 指定要索引的文檔類型為 JSON 格式
1. 指定要索引的文檔目錄
1. 指定索引的輸出目錄和名稱
1. 指定文檔索引時使用的生成器。在此情況下，使用默認的 Lucene 文檔生成器
1. 指定用於索引的線程數。在此情況下，使用單線程
1. 指定存儲索引中的位置信息、文檔向量和原始文檔的選項


In [ ]:
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input tests/resources \
  --index indexes/sample_collection_jsonl \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

2024-04-09 08:41:10,491 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:204) - Setting log level to INFO
2024-04-09 08:41:10,498 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:207) - ============ Loading Index Configuration ============
2024-04-09 08:41:10,498 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:208) - AbstractIndexer settings:
2024-04-09 08:41:10,499 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:209) -  + DocumentCollection path: tests/resources
2024-04-09 08:41:10,499 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:210) -  + CollectionClass: JsonCollection
2024-04-09 08:41:10,501 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:211) -  + Index path: indexes/sample_collection_jsonl
2024-04-09 08:41:10,501 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:212) -  + Threads: 1
2024-04-09 08:41:10,501 INFO  [main] index.AbstractIndexer (AbstractIndexer.java:213) -  + Optimize (merge segments)? false
Apr 09, 2024 8

In [ ]:
!pip install faiss-cpu
!apt-get purge openjdk-\* -qq > /dev/null
!apt-get autoremove -qq > /dev/null
!apt-get autoclean -qq > /dev/null
!apt-get install openjdk-21-jdk-headless -qq > /dev/nul
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"

In [ ]:
from pyserini.search.lucene import LuceneSearcher
import time

# 初始化 LuceneSearcher 並指定索引目錄
searcher = LuceneSearcher('indexes/sample_collection_jsonl')

start = time.time()
# 進行搜索，搜索詞為 'cancer'，返回前 K 個結果
hits = searcher.search('蝙蝠俠', k=10)
print(hits)
end = time.time()

for i in range(len(hits)):
    # 獲取文檔的檔名，此處假設檔名與文檔 ID 相同，並且位於指定目錄下的 resources 資料夾中
    filename = '/content/tests/resources/'+str(hits[i].docid)+'.json'
    f = open (filename, "r")

    data = json.loads(f.read())
    # 輸出搜索結果的文檔 ID、得分以及對應的內容
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f} \n {data}')



[<io.anserini.search.ScoredDoc at 0x7d9ec6f0ae80 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a05bfbaeb32 at 0x7d9ec7371010>>, <io.anserini.search.ScoredDoc at 0x7d9ec6f0aed0 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a05bfbaeb3a at 0x7d9ec7370990>>, <io.anserini.search.ScoredDoc at 0x7d9ec6f0af20 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a05bfbaeb4a at 0x7d9ec7370f50>>, <io.anserini.search.ScoredDoc at 0x7d9ec6f0af70 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a05bfbaeb52 at 0x7d9ec7371030>>, <io.anserini.search.ScoredDoc at 0x7d9ec6f0afc0 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a05bfbaeb5a at 0x7d9ec7370d90>>, <io.anserini.search.ScoredDoc at 0x7d9ec6f0b010 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a05bfbaeb62 at 0x7d9ec7370f10>>, <io.anserini.search.ScoredDoc at 0x7d9ec6f0b060 jclass=io/anserini/search/ScoredDoc jself=<LocalRef obj=0x5a05bfbaeb6a at 0x7d9ec7370d70>>, <io.anserini.search

In [ ]:
!ls /content/indexes/sample_collection_jsonl

_0.fdm	_0.fnm		   _0_Lucene99_0.doc  _0_Lucene99_0.tip  _0.nvm  _0.tvm      write.lock
_0.fdt	_0_Lucene90_0.dvd  _0_Lucene99_0.pos  _0_Lucene99_0.tmd  _0.si	 _0.tvx
_0.fdx	_0_Lucene90_0.dvm  _0_Lucene99_0.tim  _0.nvd		 _0.tvd  segments_1


In [32]:
from pyserini.index import IndexReader

index_path = 'indexes/sample_collection_jsonl'
index_reader = IndexReader(index_path)

term = '蝙蝠俠'
# 使用get_term_counts方法來獲取詞語的統計信息
term_counts = index_reader.get_term_counts(term, analyzer=None)
df_t = term_counts[0]  # 正確獲取document frequency

print(f"詞語 '{term}' 的文檔頻率 (df_t) 是: {df_t}")


詞語 '蝙蝠俠' 的文檔頻率 (df_t) 是: 580


In [33]:
import math

# 假設 N 為文件總數，df_t 為包含詞語t的文件數量
N = len(data) - 2

# 計算IDF
IDF_t = math.log(N / df_t)

print("蝙蝠俠的IDF值為:", IDF_t)


蝙蝠俠的IDF值為: 7.732425910876958
